In [37]:
%%file formulation.lp

#const intlen = 32.

register(eax;ebx;ecx;edx;esi;edi;edp;esp;eip;imm;cmp).
flag(n;z;p).
pos(0..intlen-1).
time(0..maxtime).
flg(0, z).
    
:- flg(T, F1), flg(T, F2), F1 != F2, flag(F1), flag(F2), time(T).

%% Registers maintain value unless acted upon
regbit(T, Reg, Pos) :-
    time(T),
    register(Reg),
    pos(Pos),
    bit(B),
    regbit(T-1, Reg, Pos),
    pcat(T, PC),
    not end(PC),
    not sub(PC, _, _, Reg),
    Reg != eip,
    %Reg != eax,
    Reg != imm.
regbit(T, imm, Pos) :-
    time(T),
    pos(Pos),
    bit(B),
    regbit(T-1, imm, Pos),
    pcat(T, PC),
    not end(PC),
    not cmp_imm(PC, _, _, _).
    
%% PC advances each time step
regbit(T+1, eip, 0) :-
    time(T),
    bit(B),
    not regbit(T, eip, 0).
regbit(T+1, eip, Pos) :-
    time(T),
    pos(Pos),
    Pos > 0,
    regbit(T, eip, Pos),
    regbit(T+1, eip, Pos-1).
regbit(T+1, eip, Pos) :-
    time(T),
    pos(Pos),
    Pos > 0,
    regbit(T, eip, Pos),
    not regbit(T, eip, Pos-1).
regbit(T+1, eip, Pos) :-
    time(T),
    pos(Pos),
    Pos > 0,
    bit(B),
    not regbit(T, eip, Pos),
    not regbit(T+1, eip, Pos-1),
    regbit(T, eip, Pos-1).
    

% Comparisons    
cmp(T) :- time(T), pcat(T, PC), cmp_imm(PC, _, _, _).
    
%% Store value from cmp_imm in imm
regbit(T, imm, Pos) :-
    time(T),
    pos(Pos),
    pcat(T, PC),
    cmp_imm(PC, _, Pos).

%% Reg - imm -> cmp
sub(PC, Reg, imm, cmp) :- cmp_imm(PC, Reg, _, _).

%% Subtraction
regbit(T, Res, Pos) :-
    time(T),
    register(Res),
    pos(Pos),
    pcat(T, PC),
    sub(PC, Reg1, Reg2, Res),
    register(Reg1),
    register(Reg2),
    regbit(T, Reg1, Pos),
    not regbit(T, Reg2, Pos),
    not carry(T, Pos).
regbit(T, Res, Pos) :-
    time(T),
    register(Res),
    pos(Pos),
    pcat(T, PC),
    sub(PC, Reg1, Reg2, Res),
    register(Reg1),
    register(Reg2),
    not regbit(T, Reg1, Pos),
    regbit(T, Reg2, Pos),
    not carry(T, Pos).
regbit(T, Res, Pos) :-
    time(T),
    register(Res),
    pos(Pos),
    pcat(T, PC),
    sub(PC, Reg1, Reg2, Res),
    register(Reg1),
    register(Reg2),
    regbit(T, Reg1, Pos),
    regbit(T, Reg2, Pos),
    carry(T, Pos).
regbit(T, Res, Pos) :-
    time(T),
    register(Res),
    pos(Pos),
    pcat(T, PC),
    sub(PC, Reg1, Reg2, Res),
    register(Reg1),
    register(Reg2),
    not regbit(T, Reg1, Pos),
    not regbit(T, Reg2, Pos),
    carry(T, Pos).
carry(T, Pos+1) :-
    time(T),
    pos(Pos),
    pcat(T, PC),
    sub(PC, Reg1, Reg2, Res),
    register(Reg1),
    register(Reg2),
    not regbit(T, Reg1, Pos),
    regbit(T, Reg2, Pos).
carry(T, Pos+1) :-
    time(T),
    pos(Pos),
    pcat(T, PC),
    sub(PC, Reg1, Reg2, Res),
    register(Reg1),
    register(Reg2),
    regbit(T, Reg1, Pos),
    regbit(T, Reg2, Pos),
    carry(T, Pos).
carry(T, Pos+1) :-
    time(T),
    pos(Pos),
    pcat(T, PC),
    sub(PC, Reg1, Reg2, Res),
    register(Reg1),
    register(Reg2),
    not regbit(T, Reg1, Pos),
    not regbit(T, Reg2, Pos),
    carry(T, Pos).
    
%% Flags
flg(T, n) :- 
    cmp(T),
    carry(T, intlen).
flg(T, z) :-
    cmp(T),
    not regbit(T, cmp, _),
    not carry(T, intlen).
flg(T, p) :-
    cmp(T),
    regbit(T, cmp, _),
    not carry(T, intlen).

Overwriting formulation.lp


In [38]:
%%file instance-6.lp

#const maxtime = 3.

cmp_imm(1, eax, 1..intlen-1, 0).
cmp_imm(1, eax, 0, 1).
end(2).
#show flg/2.

Overwriting instance-6.lp


In [36]:
!clingo formulation.lp instance-6.lp pcat8.lp -c intlen=8 --time-limit=60 --text

cmp_imm(1,eax,0,1).
end(2).
flg(0,z).
register(eax).
register(ebx).
register(ecx).
register(edx).
register(esi).
register(edi).
register(edp).
register(esp).
register(eip).
register(imm).
register(cmp).
flag(n).
flag(z).
flag(p).
bit(0).
bit(1).
pos(0).
pos(1).
pos(2).
pos(3).
pos(4).
pos(5).
pos(6).
pos(7).
regbit(0,eax,0,0).
regbit(0,ebx,0,0).
regbit(0,ecx,0,0).
regbit(0,edx,0,0).
regbit(0,esi,0,0).
regbit(0,edi,0,0).
regbit(0,edp,0,0).
regbit(0,esp,0,0).
regbit(0,eip,0,0).
regbit(0,imm,0,0).
regbit(0,cmp,0,0).
regbit(0,eax,1,0).
regbit(0,ebx,1,0).
regbit(0,ecx,1,0).
regbit(0,edx,1,0).
regbit(0,esi,1,0).
regbit(0,edi,1,0).
regbit(0,edp,1,0).
regbit(0,esp,1,0).
regbit(0,eip,1,0).
regbit(0,imm,1,0).
regbit(0,cmp,1,0).
regbit(0,eax,2,0).
regbit(0,ebx,2,0).
regbit(0,ecx,2,0).
regbit(0,edx,2,0).
regbit(0,esi,2,0).
regbit(0,edi,2,0).
regbit(0,edp,2,0).
regbit(0,esp,2,0).
regbit(0,eip,2,0).
regbit(0,imm,2,0).
regbit(0,cmp,2,0).
regbit(0,eax,3,0).
regbit(0,ebx,3,0).
regbit(0,ecx,3,0).
regbit